In [ ]:
# Install required packages (uncomment if needed)
# !pip install oci openlayer


In [ ]:
import oci
from oci.generative_ai_inference import GenerativeAiInferenceClient
from oci.generative_ai_inference.models import (
    ChatDetails,
    GenericChatRequest,
    Message,
    OnDemandServingMode
)

# Import the Openlayer tracer
from openlayer.lib.integrations import trace_oci_genai


In [ ]:
# Configuration - Update these values for your environment
COMPARTMENT_ID = "your-compartment-ocid-here"  # Replace with your compartment OCID
ENDPOINT = "https://inference.generativeai.us-chicago-1.oci.oraclecloud.com"  # Replace with your region's endpoint

# Load OCI configuration
config = oci.config.from_file()  # Uses default config file location
# Alternatively, you can specify a custom config file:
# config = oci.config.from_file("~/.oci/config", "DEFAULT")

# Create the OCI Generative AI client
client = GenerativeAiInferenceClient(
    config=config,
    service_endpoint=ENDPOINT
)

print("✅ OCI Generative AI client initialized")


In [ ]:
# Apply Openlayer tracing to the OCI client
traced_client = trace_oci_genai(client)

print("✅ Openlayer tracing enabled for OCI Generative AI client")


In [ ]:
# Create a chat request
chat_request = GenericChatRequest(
    messages=[
        Message(
            role="user",
            content="Hello! Can you explain what Oracle Cloud Infrastructure is?"
        )
    ],
    # Available models (choose one):
    # - "cohere.command-r-16k"
    # - "cohere.command-r-plus"
    # - "meta.llama-3.1-70b-instruct"
    # - "meta.llama-3.1-405b-instruct"
    model_id="cohere.command-r-plus",
    max_tokens=200,
    temperature=0.7,
    is_stream=False  # Non-streaming
)

chat_details = ChatDetails(
    compartment_id=COMPARTMENT_ID,
    chat_request=chat_request
)

print("🚀 Making non-streaming chat completion request...")

# Make the request with custom inference ID for tracking
response = traced_client.chat(
    chat_details,
    inference_id="oci-example-1-non-streaming"
)

print("✅ Response received:")
print(f"Model: {response.data.model_id}")
print(f"Content: {response.data.choices[0].message.content}")
print(f"Tokens used: {response.data.usage.prompt_tokens} prompt + {response.data.usage.completion_tokens} completion = {response.data.usage.total_tokens} total")


In [ ]:
# Create a streaming chat request
streaming_chat_request = GenericChatRequest(
    messages=[
        Message(
            role="system",
            content="You are a helpful AI assistant that provides concise, informative answers."
        ),
        Message(
            role="user",
            content="Tell me a short story about cloud computing and AI working together."
        )
    ],
    model_id="meta.llama-3.1-70b-instruct",
    max_tokens=300,
    temperature=0.8,
    is_stream=True  # Enable streaming
)

streaming_chat_details = ChatDetails(
    compartment_id=COMPARTMENT_ID,
    chat_request=streaming_chat_request
)

print("🚀 Making streaming chat completion request...")
print("📡 Streaming response:")
print("-" * 50)

# Make the streaming request
streaming_response = traced_client.chat(
    streaming_chat_details,
    inference_id="oci-example-2-streaming"
)

# Process the streaming response
full_content = ""
for chunk in streaming_response:
    if hasattr(chunk, 'data') and hasattr(chunk.data, 'choices'):
        if chunk.data.choices and hasattr(chunk.data.choices[0], 'delta'):
            delta = chunk.data.choices[0].delta
            if hasattr(delta, 'content') and delta.content:
                print(delta.content, end='', flush=True)
                full_content += delta.content

print("\n" + "-" * 50)
print("✅ Streaming completed!")
print(f"📊 Total content length: {len(full_content)} characters")


In [ ]:
# Advanced parameters example
advanced_request = GenericChatRequest(
    messages=[
        Message(
            role="user",
            content="Write a creative haiku about artificial intelligence."
        )
    ],
    model_id="meta.llama-3.1-70b-instruct",
    max_tokens=100,
    temperature=0.9,  # High creativity
    top_p=0.8,
    frequency_penalty=0.2,  # Reduce repetition
    presence_penalty=0.1,
    stop=["\n\n"],  # Stop at double newline
    is_stream=False
)

advanced_details = ChatDetails(
    compartment_id=COMPARTMENT_ID,
    chat_request=advanced_request
)

print("🚀 Making request with advanced parameters...")

try:
    response = traced_client.chat(
        advanced_details,
        inference_id="oci-example-3-advanced-params"
    )
    
    print("✅ Creative response received:")
    print(f"{response.data.choices[0].message.content}")
    print(f"\n📊 Parameters used:")
    print(f"- Temperature: 0.9 (high creativity)")
    print(f"- Top-p: 0.8")
    print(f"- Frequency penalty: 0.2")
    print(f"- Presence penalty: 0.1")
    
except Exception as e:
    print(f"❌ Error occurred: {type(e).__name__}: {str(e)}")
    print("✅ Error was properly caught and traced by Openlayer")
